In [1]:
from joblib import Parallel, delayed
from optimize import *
import time
cmap = plt.get_cmap('viridis')


In [2]:
configuraciones = ['--cr un --mu bf --a ur --rep rm --pop 200 --dec 50 --prob_cr 0.5 --prob_mu 0.25 --prob_ini 0.1',
                    '--cr un --mu bf --a ur --rep rm --pop 200 --dec 50 --prob_cr 0.75 --prob_mu 0.5 --prob_ini 0.25',
                    '--cr hun --mu bfo --a ur --rep rm --pop 200 --dec 50 --prob_cr 0.5 --prob_mu 0.5 --prob_ini 0.25',
                    '--cr un --mu bf --a ur --rep rm --pop 200 --dec 50 --prob_cr 0.5 --prob_mu 0.5 --prob_ini 0.1',
                    '--cr un --mu bfo --a ur --rep rm --pop 200 --dec 50 --prob_cr 0.5 --prob_mu 0.5 --prob_ini 0.1']

In [3]:
instance_id = '7'
instance = '../GeoData/Instances/Instance_' + instance_id + '.pkl'

output = 'Results/testPerformance'+instance_id
testPerformance = dict([i, {'hv':[], 'res':[], 'time':[]}] for i in range(len(configuraciones)))
plot_performance = False
plot_time = False

## Celda para ejecutar el análisis desde cero
En esta celda se realizan las 20 ejecuciones de cada configuración para la instancia establecida arriba

In [4]:
if False:
    for i,p in enumerate(configuraciones):
        
        for ejec in range(20):
            seed = str(ejec*308)
            
            # Extra whitespace around options is important!
            parameters = ' -i ' + instance + ' --seed ' + seed + ' ' + p

            ini = time.time()
            hv, res = exec_algorithm(parameters, returnData=True, timeout=10)
            fin = time.time()
            
            testPerformance[i]['hv'].append(hv)
            testPerformance[i]['res'].append(res)
            testPerformance[i]['time'].append(fin - ini)
            
            finish = ['ok','fail'][res == -1]
            
            linea = str(i) + ' ' + str(ejec) + ' ' + str(hv) + ' ' + str(fin - ini) + ' ' + finish + '\n'
            with open(output + '.txt', 'a') as file:
                file.write(linea)
                
            with open(output+'.pkl', 'wb') as f:  
                pickle.dump(testPerformance, f)
                f.close()
            
            print(linea)

## Celda para cargar el análisis previo
PAra no volver a ejecutar desde cero el análisis, se carga la información del archivo pickle.

In [5]:
with open('Results/testPerformance' + instance_id+ '.pkl', 'rb') as f:
    testPerformance = pickle.load(f) 
    f.close()

### Cargar datos de irace
En caso de ser de utilidad

In [6]:
irace = pd.read_csv('../ajuste/export/export.csv')
irace.pop('Unnamed: 0')

ejecs = irace[irace['instancename'] == 'Instance_' + instance_id]
ejecs = ejecs[ejecs['type']=='regular']
ejecs = ejecs[ejecs['value'] <= 0.99]

In [7]:
otras_conf = ejecs['value'].to_list()
otras_conf_color = ejecs['id'].to_list()
bkv = ejecs['bkv'].to_list()[0]

## Graficar el desempeño por instancia

In [8]:
if plot_performance:
    fig, ax = plt.subplots(figsize=(3, 8))

    # Coordenadas x para las barras
    ind = len(testPerformance)

    # Ancho de las barras
    ancho_barra = 0.5

    hipervolumenes = [testPerformance[i]['hv'] for i in range(ind)]

    tiempos = [np.mean(testPerformance[i]['time']) for i in range(ind)]

    ax.boxplot(hipervolumenes,widths=ancho_barra, showfliers=True, zorder = 3, 
            medianprops=dict(color=(1,0,0,1)), )
    # ax.scatter([0]*len(otras_conf), otras_conf, c=otras_conf_color, cmap='viridis', 
    #            marker='s', zorder = 3, alpha=0.2, linewidths=0, label='Configuraciones regulares')
    for i in range(ind):
        ax.scatter([i+1]*len(testPerformance[i]['hv']), testPerformance[i]['hv'], 
                cmap='viridis', marker='x', zorder = 3, alpha=0.2, linewidths=1)

    ax.plot([-1,6], [bkv,bkv], '--', color='black', zorder = 0, label='Mejor valor conocido')

    # Configuración de ejes y etiquetas
    ax.set_xlabel('$Configuración$')
    ax.set_ylabel('$Hipervolumen$')
    ax.set_title('$Instancia\ ' + instance_id + '$')
    ax.set_xticklabels([f'{i+1}' for i in range(ind)])
    ax.set_xlim([0,6])

    # Leyenda para identificar los modelos
    ax.legend()
    ax.grid(True,zorder=0, color='gray', linestyle='--', linewidth=0.4)
    ax.set_ylim([bkv-0.005, np.max(hipervolumenes)+0.01])

    fig.savefig('/home/cic/Tesis/Figuras/Results/testPerformance' + instance_id + '.png', dpi=300, bbox_inches='tight')

In [61]:
insts = ['4','7','5']
tablas_20 = dict()
pf_cmap = plt.get_cmap('viridis')

for num,instance_id in enumerate(insts):
    instance = '../GeoData/Instances/Instance_' + instance_id + '.pkl'
    

    with open('Results/testPerformance' + instance_id + '.pkl', 'rb') as f:
        testPerformance = pickle.load(f) 
        f.close()
    hipervolumenes = [testPerformance[i]['hv'] for i in range(len(testPerformance))]
    min_hv = np.min(hipervolumenes,axis = 1)
    max_hv = np.max(hipervolumenes,axis = 1)
    mean_hv = np.mean(hipervolumenes,axis = 1)
    median_hv = np.median(hipervolumenes,axis = 1)
    std_hv = np.std(hipervolumenes,axis = 1)
    
    data_20 = dict()
    data_20['config'] = [i for i in range(len(configuraciones))]
    data_20['min'] = min_hv
    data_20['máx'] = max_hv
    data_20['prom'] = mean_hv
    data_20['med'] = median_hv
    data_20['desv'] = std_hv
    
    data_20 = pd.DataFrame(data_20, index = data_20['config'])
    
    tablas_20[instance_id] = data_20
    
    

In [66]:
tablas_20['5']

,config,min,máx,prom,med,desv
0,0,0.438961,0.482117,0.456855,0.458180,0.014045
1,1,0.439624,0.481920,0.453589,0.446290,0.014507
2,2,0.440634,0.461759,0.448708,0.445693,0.007023
3,3,0.437618,0.503250,0.454842,0.457937,0.013576
4,4,0.429089,0.479557,0.449520,0.445507,0.011229


## Graficar el tiempo de ejecución para todas las instancias de prueba

In [9]:
if plot_time:
    insts = ['4','7','5']
    tiempos = dict([i, []] for i in insts)
    num_confs = len(configuraciones)
    ind = np.arange(num_confs)
    
    for i in insts:
        with open('Results/testPerformance' + i+ '.pkl', 'rb') as f:
            testPerformance = pickle.load(f) 
            tiempos[i] = [testPerformance[j]['time'] for j in range(len(testPerformance))]
            f.close()


    
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Ancho de las barras
    ancho_barra = 1
    esp = 0.15 # Espacio entre barras

    # Dibujar las barras
    rects1 = ax.bar(2*ind + ind*3 - 0.5, np.mean(tiempos['4'],axis=1), ancho_barra, label='$Instancia\ 4\ (|J|=19)$', color=cmap(0.2), zorder = 2)
    rects2 = ax.bar(2*ind + ind*3 + 1 - 0.5, np.mean(tiempos['7'],axis=1), ancho_barra, label='$Instancia\ 7\ (|J|=37)$', color=cmap(0.5), zorder = 2)
    rects3 = ax.bar(2*ind + ind*3 + 2 - 0.5, np.mean(tiempos['5'],axis=1), ancho_barra, label='$Instancia\ 5\ (|J|=53)$', color=cmap(0.8), zorder = 2)

    # ax.plot([-10,100],[np.mean(promedio_info), np.mean(promedio_info)], color=cmap(0.2), linestyle='--', linewidth=1, zorder=2)
    # ax.plot([-10,100],[np.mean(promedio_procesos), np.mean(promedio_procesos)], color=cmap(0.5), linestyle='--', linewidth=1, zorder=2)

    # Configuración de ejes y etiquetas
    ax.set_xlabel('$Configuración$')
    ax.set_ylabel('$Tiempo (s)$')
    ax.set_title('Tiempo de procesamiento de las instancias')
    ax.set_xticks([i*2 + 3 * i + 0.5 for i in range(len(tiempos['5']))])
    ax.set_xticklabels([f'{i+1}' for i in range(len(tiempos['5']))])
    ax.set_xlim([-2, 2*(num_confs-1) + 3*(num_confs-1) +3])
    ax.set_ylim([0, 1.6*np.max([np.mean(tiempos['4']), np.mean(tiempos['7']), np.mean(tiempos['5'])])])

    # Leyenda para identificar los modelos
    ax.legend()
    ax.grid(True,zorder=0, color='gray', linestyle='--', linewidth=0.4)
    # Mostrar la gráfica
    fig.savefig('/home/cic/Tesis/Figuras/Results/tiemposOptimización.png', dpi=300)

# Obtiene el historial del frente de Pareto para la mediana

In [10]:
# Para ejecutar desde cero
test = False

In [11]:
# Función para paralelizar
def optimiza(parameters, save=True, path='Results/', name='resultado',sh=False):
    hv,res = exec_algorithm(parameters, returnData=True, timeout=10,sh=sh)
    if save:
        with open(path+name+'.pkl', 'wb') as f:  
                    pickle.dump(res, f)
                    f.close()
        #print(name, hv, res)
    else:
        #print(name, hv, res)
        return name, hv, res

In [12]:
insts = ['4','7','5']
#insts = ['5']
medianas = dict([i, []] for i in insts)

for i in insts:
    with open('Results/testPerformance' + i+ '.pkl', 'rb') as f:
        testPerformance = pickle.load(f) 
        f.close()
    
    hipervolumenes = [testPerformance[i]['hv'] for i in range(len(testPerformance))]
    
    mediana = np.argsort(hipervolumenes, axis=1)
    mediana = mediana[:,len(mediana[0])//2]
    
    medianas[i] = mediana
    
ejec_medianas = dict([i, dict([(j,'') for j in range(len(configuraciones))])] for i in insts)

ejecs = []
for instance_id in insts:
    instance = '../GeoData/Instances/Instance_' + instance_id + '.pkl'
    
    for i,p in enumerate(configuraciones):
        seed = medianas[instance_id][i]*308
        
        parameters = ' -i ' + instance + ' --seed ' + str(seed) + ' ' + p
        ejecs.append([instance_id,i,parameters])


In [13]:
if test:
    Parallel(n_jobs=3)(delayed(optimiza)(e[2],name=str(e[0])+'_'+str(e[1]),sh=True,save=True) for e in ejecs)
#resultados = [optimiza(e[0],e[1],e[2]) for e in ejecs]

# Verifica la mediana
# for i,m in enumerate(medianas['5']):
#     print(testPerformance[i]['hv'][m])

/home/cic/anaconda3/envs/rapids-pymoo/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


# Convergencia del frente de Pareteo de la mediana

In [ ]:

pf_cmap = plt.get_cmap('viridis')

for num,instance_id in enumerate(insts):
    instance = '../GeoData/Instances/Instance_' + instance_id + '.pkl'
    
    
    for conf,p in enumerate(configuraciones):
    
        with open('Results/' + str(instance_id) + '_' + str(conf) + '.pkl', 'rb') as f:
            res = pickle.load(f) 
            f.close()
        
        instance = '../GeoData/Instances/Instance_' + instance_id + '.pkl'
        with open(instance, 'rb') as f:
            instance = pickle.load(f) 
            f.close()
        
        capacidades = instance['capacidades']
        demanda = instance['demanda']
        
        max_num_bases = np.ceil(len(demanda)/2)
        max_f1, max_f2 = [len(demanda)*9000, max_num_bases + 1]

        pf = copy(res.F)
        pf[:,0] = pf[:,0]/max_f1
        pf[:,1] = pf[:,1]/max_f2
        pf = pf[pf[:,1].argsort()]
        ind = HV(ref_point=[1,1])
        print(instance_id,conf,"HV", 1-ind(pf))
        
        fig,axs = plt.subplots(1,1, figsize=(4,4), constrained_layout=True)

        transp = lambda x: (0.7,0,0.6,0.1)


        for id,r in enumerate(res.history):
            i = id/len(res.history)
            pfi = copy(r.pop.get("F"))
            pfi[:,0] = pfi[:,0]/max_f1
            pfi[:,1] = pfi[:,1]/max_f2
            
            indices_sort = np.argsort(pfi[:, 1])
            pfi = pfi[indices_sort]
            
            viol = copy(r.pop.get("CV"))
            viol = viol[indices_sort]
            

            
            colores_relleno = [pf_cmap(i) if j == 0 else transp(pf_cmap(i)) for j in viol]
            tamano = [30 if j == 0 else 15 for j in viol]
            
            axs.scatter(pfi[:, 1]*max_f2, pfi[:, 0]*max_f1, s=tamano, facecolors=colores_relleno, 
                        edgecolors='none',zorder=2)
            

            # if id == 0:
            #     axs[0].scatter(pfi[:, 1]*max_f2, pfi[:, 0]*max_f1, marker = 'x', color='r',zorder=0, label='Solucion inicial')
            
        axs.plot(pf[:, 1]*max_f2, pf[:, 0]*max_f1, color='r',zorder=3, lw=0.5, ls='--')
        axs.scatter(pf[:, 1]*max_f2, pf[:, 0]*max_f1, color='r',zorder=3, lw=0, s=10,label='Frente final')

        axs.set_box_aspect(1)
        axs.set_xlabel('$f_2$')
        axs.set_ylabel('$f_1$')
        axs.set_xlim([0,max_f2])
        axs.set_ylim([0,max_f1])
        axs.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        step = [1,2][(max_f2 > 20)*1]
        axs.set_xticks([i for i in range(0,int(max_f2)+1,step)])
        axs.set_title('Instancia ' + instance_id + ' - Configuración ' + str(conf+1))
        axs.grid(True,zorder=0, color='gray', linestyle='--', linewidth=0.2)
        #axs.legend()
        fig.savefig('/home/cic/Tesis/Figuras/Results/pareto_' + instance_id + '_' + str(conf) + '.png', dpi=300)
        

# Convergencia del frente de Pareto de las 

In [45]:
#testPerformance[0]['hv'][medianas[instance_id][0]]